In [23]:
from nltk import sent_tokenize, pos_tag
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from string import punctuation
from IPython.display import display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [2]:
def penn_to_wn(tag):
    """
        Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [14]:
def get_sentiment_score(text):
    
    """
        This method returns the sentiment score of a given text using SentiWordNet sentiment scores.
        input: text
        output: numeric (double) score, >0 means positive sentiment and <0 means negative sentiment.
    """    
    total_score = 0
    #print(text)
    raw_sentences = sent_tokenize(text)
    #print(raw_sentences)
    
    for sentence in raw_sentences:

        sent_score = 0     
        sentence = str(sentence)
        #print(sentence)
        sentence = sentence.replace("<br />"," ").translate(str.maketrans('','',punctuation)).lower()
        tokens = TreebankWordTokenizer().tokenize(text)
        tags = pos_tag(tokens)
        for word, tag in tags:
            wn_tag = penn_to_wn(tag)
            if not wn_tag:
                continue
            lemma = WordNetLemmatizer().lemmatize(word, pos=wn_tag)
            if not lemma:
                continue
            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())
            sent_score += (swn_synset.pos_score() - swn_synset.neg_score())

        total_score = total_score + 1 if sent_score>0 else total_score - 1

    
    return total_score

In [4]:
import pandas as pd
import numpy as np
from IPython.display import display
pd.set_option('display.max_columns', None)

In [5]:
reviews = pd.read_csv("../data/small_corpus.csv")

In [10]:
reviews.shape

(4500, 12)

In [6]:
reviews.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
0,1.0,True,"11 30, 2015",A3AC92K59QLYR8,B00503E8S2,ben,Game freezes over and over its unplayable,it just doesn't work,1448841600,NaN,{'Format:': ' Video Game'},NaN
1,1.0,False,"05 19, 2012",A334LHR8DWARY8,B00178630A,Xenocide,I have no problem with needing to be online to...,The only real way to show Blizzard our feeling...,1337385600,23,{'Format:': ' Computer Game'},NaN
2,1.0,True,"10 19, 2014",A28982ODE7ZGVP,B001AWIP7M,Eric Frykberg,NOT GOOD,One Star,1413676800,NaN,{'Format:': ' Video Game'},NaN
3,1.0,True,"09 6, 2015",A19E85RLQCAMI1,B00NASF4MS,Joe,Really not worth the money to buy this game on...,Really not worth the money to buy this game on...,1441497600,2,{'Format:': ' Video Game'},NaN
4,1.0,False,"05 28, 2008",AEMQKS13WC4D2,B00140P9BA,Craig,They need to eliminate the Securom. I purchase...,Securom can ruin a great game,1211932800,55,{'Format:': ' DVD-ROM'},NaN


In [12]:
reviews.dropna(subset=['reviewText'], inplace=True)

In [13]:
reviews.shape

(4499, 12)

In [15]:
reviews['sentiment'] = reviews['reviewText'].apply(lambda text : get_sentiment_score(text))

In [24]:
reviews[['reviewText','sentiment']].sample(5)

,reviewText,sentiment
3239,loved it,1
2995,"Sale buy. I got it for $15, was worth that at least. Don't expect much; it's an EA game.",3
1508,"The game was good for the first 60 levels, then it became clear that Jay Wilson and the team had slighted the majority of the die hard Diablo fans. It's a year after the game was released and it's still pretty much in beta. It's kind of sad, actually. The game was worth the money I paid (I suppose) because I got plenty of game hours in. It's just not the Diablo game I expected. I will wait for it to become what it could be, but I'm skeptical.",-6
2048,"God of War and God of War 2, originally for the PS2, are both present in this game. I'll get to the games themselves later. First, however, is the presentation, which is the main draw of this collection. The games are remastered in high definition during the gameplay sections, which look fantastic, but the cutscenes are not remastered and are of noticeably lower quality. However, this doesn't detract from the overall quality. There is added trophy support, which are a bit on the easy side. Most of the trophies are story-related, however other trophies include collecting bonus items and beating challenges. Basically, if you are fans of these games, you will enjoy this presentation, so stop reading and go buy it (or, buy the 5 game God of War saga collection) However, for newcomers, here is my opinion of the game:\n\nYou play as Kratos, a warrior with a troubled past. Your quest consists of Kratos trying to appease the gods so they will rid him of the memories that trouble him. Throughout this quest, you basically go down hallways and through rooms killing seemingly endless swarms of monsters. You have a few weapons and powers to use in an attempt to fight off your enemies. Occasionally, you come across a puzzle, which usually require pushing objects around, though there are a few others. Some quick-time events where you have to push a button displayed on the screen during the cutscene. There isn't much more to the game than going from room to room fighting the same few enemies over and over again with a puzzle here or there, so if that sounds like fun to you, by all means, get it. I find doing the same thing in every single room to be dull.\n\nOverall, if you enjoy hack n slash action games, then pick up either this collection or the God of War Saga collection. They are easily the best version of these games. However, other HD collections that came later have better presentation (namely HD cutscenes) and more varied gameplay, so I give the collection 3 stars.",18
1713,I got this thinking it would cure my Sim City burn out from previous versions. It didn't. It's still a decent kids game.,3


In [25]:
reviews[['reviewText','sentiment']].sample(5)

,reviewText,sentiment
2517,"It seems to me that if you have to qualify an RPG as a Japanese RPG on the XBox 360 to apply the word 'best' then somehow the point has been missed. So before I give what will really be a positive review let me point out that Magna Carta II is not the best RPG on the XBox 360, and arguably not the best JRPG either. And if the rushes for FF XIII are any indicator, even if Magna Carta II was a 'best' it wouldn't be so for very long. All that being said, MC2 is actually a good game with a lot going for it and a few things going against it.\n\nThe plot is intricate - Juto, a boy who has lost his memory is building a life on an island. His friend (and boss) is Melissa, who is charged with the defense of the island. A revolution is in process and Juto finds himself thrust between opposing forces as the war overtakes the peaceful villagers. Tragedy strikes and Juto chooses his side (the Southern forces) and swears revenge. He meets his teamates - spell casters and combate specialists - and the game is off and running. The female lead is Zephie (Princess Rzephilda) who is trying to regain the kingdom she lost when the inevitable, evil prime miniter kills her mother.\n\nThe plot is complex, but fairly linear. There are any number of quests to clear to provide breaks, and you can often get away for some level building. However, for all the twists and turns, progress is often forced. And the quest structure does not always harmonize with the main story. The characters take a while to grow on you. All of them are irritating when they start out and they can take over half of the game to become likeable. Still the story tries to show an occasional bit of originality and captures the players interest, which is a vast improvement over a few other games.\n\nThe artwork and music are several steps above average. Lots of good CGI clips, the use of several styles to mark differences in narrative, and good looking characters make up the visual environment. Characters actually have gestures and expressions. The battle artwork is a little less satisfactory but is almost good enough. Occasionally I found it hard to make out what was what in a busy scene. Like when you are trying to pick off the healers in a crowd of enemies. But that's the only real flaw, and the backgrounds and buildings ate very nicely don't. This is a good game to look at.\n\nIt is a less good game to play in when you find yourself in battle. Magna Carta II has one of those inordantly complex fight systems that take forever to adjust to and send fingers flying over the controller. You are perpetual shifting characters, changing spells and attacks, and working up to combinations. Usually by the time you get a character lined up to deliver the final blow the freeze from exhaustion and you have to bring up andther character and fit them in as well. There never seems to be a rhythm to the battle, and if I were to pick out the game's worst flaw this would be it.\n\nAnother issue I have is being dragged thru conflicts with little or no access to a save area. This doesn't always happen, but it's tedious to have to run back and several scenes in order to save and then run forward again in order to slay more enemies. Over all, though, there are more good things than bad in Magna Carta II. If you like intricate plots, curvacious heroines, and frantic battles you will find the game to your liking and I'd happily recommend it to RPG players",34
3178,"An outstanding game that fit my tastes perfectly.\nAmazing music, story line and everything. It may not have the depth for some people, but I got many hours of enjoyment out of it. There's a large amount of dialogue and little cutscenes that are fully voice acted by a cast of famous Japanese actors that you may recognize. By some people's standards, the game isn't graphically impressive, but I'm not one for realism.",5
1214,"I am so very disappointed in these.\n-Mic is very quiet, even at 100% (where the PS4 says you'r